# investment and market intelligence

strategic analysis for property investment decisions in ibadan

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter
import warnings
warnings.filterwarnings('ignore')

plt.style.use('default')
sns.set_palette("Set1")
plt.rcParams['figure.figsize'] = (14, 10)

df = pd.read_csv('../data/ibadan_housing_prices.csv')
print(f"analyzing investment opportunities across {len(df):,} properties")

## market segmentation analysis

In [ ]:
# define price segments
def categorize_price(price):
    if price < 20e6:
        return 'budget (< ₦20M)'
    elif price < 50e6:
        return 'affordable (₦20-50M)'
    elif price < 100e6:
        return 'mid-market (₦50-100M)'
    elif price < 200e6:
        return 'premium (₦100-200M)'
    else:
        return 'luxury (> ₦200M)'

df['price_segment'] = df['price_naira'].apply(categorize_price)

# market segmentation analysis
segment_analysis = df.groupby('price_segment').agg({
    'price_naira': ['count', 'mean', 'median'],
    'area_sqm': 'mean',
    'bedrooms': 'mean',
    'desirability_score': 'mean',
    'security_rating': 'mean'
}).round(2)

segment_analysis.columns = ['count', 'mean_price', 'median_price', 
                           'avg_area', 'avg_bedrooms', 'avg_desirability', 'avg_security']

# order segments by price
segment_order = ['budget (< ₦20M)', 'affordable (₦20-50M)', 'mid-market (₦50-100M)', 
                'premium (₦100-200M)', 'luxury (> ₦200M)']
segment_analysis = segment_analysis.reindex(segment_order)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# market share by segment
segment_counts = df['price_segment'].value_counts().reindex(segment_order)
colors = ['lightcoral', 'gold', 'lightblue', 'lightgreen', 'plum']
wedges, texts, autotexts = axes[0,0].pie(segment_counts.values, labels=segment_counts.index, 
                                         autopct='%1.1f%%', colors=colors, startangle=90)
axes[0,0].set_title('market share by price segment')

# average characteristics by segment
x_pos = np.arange(len(segment_analysis))
width = 0.35

bars1 = axes[0,1].bar(x_pos - width/2, segment_analysis['avg_area'], width, 
                     label='avg area (sqm)', alpha=0.8, color='steelblue')
bars2 = axes[0,1].bar(x_pos + width/2, segment_analysis['avg_bedrooms']*50, width, 
                     label='avg bedrooms (x50)', alpha=0.8, color='orange')

axes[0,1].set_xticks(x_pos)
axes[0,1].set_xticklabels([seg.split('(')[0].strip() for seg in segment_analysis.index], rotation=45)
axes[0,1].set_ylabel('value')
axes[0,1].set_title('property characteristics by segment')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# price distribution within segments
segment_order_for_plot = df.groupby('price_segment')['price_naira'].median().sort_values().index
df_ordered = df.copy()
df_ordered['price_segment'] = pd.Categorical(df_ordered['price_segment'], 
                                           categories=segment_order_for_plot, ordered=True)

sns.violinplot(data=df_ordered, x='price_segment', y='price_naira', ax=axes[1,0])
axes[1,0].set_xticklabels([seg.split('(')[0].strip() for seg in segment_order_for_plot], rotation=45)
axes[1,0].set_ylabel('price (₦)')
axes[1,0].set_title('price distribution within segments')
axes[1,0].yaxis.set_major_formatter(FuncFormatter(lambda x, p: f'₦{x/1e6:.0f}M'))

# quality vs price by segment
for i, segment in enumerate(segment_order):
    segment_data = df[df['price_segment'] == segment]
    if len(segment_data) > 0:
        axes[1,1].scatter(segment_data['desirability_score'], segment_data['price_naira']/1e6, 
                         label=segment.split('(')[0].strip(), alpha=0.6, s=30, color=colors[i])

axes[1,1].set_xlabel('desirability score')
axes[1,1].set_ylabel('price (millions ₦)')
axes[1,1].set_title('desirability vs price by segment')
axes[1,1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# segment summary
print("market segmentation insights:")
for segment in segment_order:
    if segment in segment_analysis.index:
        data = segment_analysis.loc[segment]
        percentage = (data['count'] / len(df)) * 100
        print(f"\n{segment}:")
        print(f"  market share: {percentage:.1f}% ({data['count']} properties)")
        print(f"  median price: ₦{data['median_price']/1e6:.1f}M")
        print(f"  typical size: {data['avg_area']:.0f} sqm, {data['avg_bedrooms']:.1f} bedrooms")
        print(f"  quality level: {data['avg_desirability']:.1f}/5.0 desirability")